<a href="https://colab.research.google.com/github/assafbz/HebrewNLP/blob/master/SpacyHebrewNer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Tagged Corpus
Downloaded from https://www.cs.bgu.ac.il/~elhadad/nlpproj/naama/ Naama Ben-Mordecai's MSc thesis and work on HebNer

In [1]:
!pip install wget
import wget
url = 'https://www.cs.bgu.ac.il/~elhadad/nlpproj/naama/tagged_corpus.txt'
wget.download(url)

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=899e6a147e27cfc4e2dfaee8a84fc83a1181d72d88fac583b198fd653242fb38
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


'tagged_corpus.txt'

# Read and manipulate train data for SpaCy
note that text are rebuilt from tagged corpus cause no original texts were supplied

In [2]:
train_data = []
entities = []
text = ""
offset = 0
is_first = True
with open('tagged_corpus.txt', 'r') as f:
    line = f.readline()
    while line:
        if line.startswith("--DOCSTART") and is_first == False:
            train_data.append((text, {"entities": entities}))
            entities = []
            text = ""
            offset = 0
        else:
            vals = line.split(' ')
            if len(vals) > 1:
                token,tag = line.split(' ')
                tag = tag.rstrip()
                if tag != 'O':
                    entity = (offset, offset + len(token), tag)
                    entities.append(entity)
                text = text + token + " "
                offset += len(token) + 1
        is_first = False
        line = f.readline()
        
print("Retrieved", len(train_data), "Samples")

Retrieved 189 Samples


# Check train data is correct and offsets are aligned

In [3]:
sample = train_data[1]
for entity in sample[1]["entities"]:
    print(sample[0][entity[0]:entity[1]], entity[2])

ישראלים I_MISC__AFF
14 I_PERCENT
אחוז I_PERCENT
3 I_PERCENT
% I_PERCENT
הישראליות I_MISC__AFF
55 I_PERCENT
% I_PERCENT
מהישראליות I_MISC__AFF
43 I_PERCENT
% I_PERCENT
קלוגס I_ORG
חקר I_ORG
רייטינג I_ORG
6 I_PERCENT
% I_PERCENT
מהישראלים I_MISC__AFF
1 I_PERCENT
% I_PERCENT
ואחוז I_PERCENT


# Get Labels
Keep only distinct labels

In [4]:
all_labels = []
for doc in train_data:
    for entity in doc[1]['entities']:
        all_labels.append(entity[2])
labels = list(set(all_labels))
labels

['I_TIME',
 'I_DATE',
 'B_PERS',
 'I_MONEY',
 'I_PERS',
 'I_MISC__ENT',
 'I_MISC__AFF',
 'B_LOC',
 'B_DATE',
 'I_LOC',
 'B_MISC__AFF',
 'B_TIME',
 'I_PERCENT',
 'I_ORG',
 'B_ORG',
 'B_MISC__ENT',
 'I_MISC_EVENT']

# Build Spacy Hebrew Ner Model
basic training

In [5]:
import spacy
import random
spacy.require_gpu()
nlp = spacy.blank("he")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner, last=True)

random.shuffle(train_data)
for label in labels:
    ner.add_label(label)
optimizer = nlp.begin_training()
for text, annotations in train_data:
    nlp.update([text], [annotations])

# Test On Train Data Sample

In [8]:
test_text = train_data[1][0]
print(test_text)
print('==========================')
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

גלישה נוספת של מי קולחין מטוהרים למימי מפרץ אילת גרמה אמש לזיהום המפרץ וליצירת כתם בוצה ענקי סמוך לגבול היבשתי עם ירדן . הדבר אירע לאחר שסוללות העפר , שהוקמו יממה קודם לכן , בכדי למנוע את גלישלת מי קולחין המטוהרים לים , קרסו והביאו לגלישת המים ממאגר אילת , לשטחים חקלאיים ולים . ממשרד הבריאות נמסר , כי מדובר במי קולחין ברמת מליחות גבוהה , שעברו טיהור ונמצא כי רמת חיידקי הקולי צואתי במים אלו נמוכה מהרגיל ואינה מסכנת בני אדם . מיד לאחר שהתגלה כתם הבוצה , הופסקה הזרמת מי הקולחין ממאגר אילת לים , והמים הוזרמו לתעלת הקינט ומשם לבריכות השיקוע של עיריית אילת הממוקמות בערבה . 
I_LOC מפרץ
I_LOC אילת
I_ORG המפרץ
I_LOC אילת
I_ORG ממשרד
I_ORG הבריאות
I_LOC אילת
I_ORG עיריית
I_LOC בערבה


# Test On New Sample

In [9]:
test_text = "כבר שנים רבות שמשרד הבריאות אינו מאפשר לאזרחי ישראל לבצע פעילות בסיסית"
print(test_text)
print('==========================')
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

כבר שנים רבות שמשרד הבריאות אינו מאפשר לאזרחי ישראל לבצע פעילות בסיסית
I_LOC ישראל
